# Mnist_Resnet101

## Import

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers as KL
from tensorflow.keras import models as KM
from tensorflow.keras import losses as LOSS
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## load Dataset mnist

In [7]:
train, test = mnist.load_data()
train_data = train[0]
train_label = train[1]
test_data = test[0]
test_label = test[1]

## load ResNet101
dataset for mnist size, [28, 28, 1]

In [8]:
resnet = ResNet101(weights=None, input_tensor=KL.Input(shape=[28,28,1]))
# customized output
x = resnet.output
x = KL.Dense(10)(x)
model = KM.Model(inputs=resnet.input, outputs=x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 34, 34, 1)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 14, 14, 64)   3200        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 14, 14, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

## Define callbacks

In [10]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0),
    ModelCheckpoint(filepath='../temp_model/model.h5', verbose=1, save_best_only=True),
]

## Fitting models

In [15]:
model.compile(
    optimizer="adam",
    loss=LOSS.sparse_categorical_crossentropy,
    metrics=["acc"])

model.fit(
    train_data, 
    train_label,
    epochs=3,
    verbose=1,
    callbacks=callbacks,
    validation_split=0.1
)

## Predict

In [19]:
model = KM.load_model("../temp_model/model.h5")
pred = model.predict(test_data, verbose=1)

313/313 [==============================] - 30s 97ms/step


In [24]:
pred = np.argmax(pred, axis=1)

In [27]:
accuracy = (test_label == pred).sum() / pred.shape[0]
print(f"accuracy:{accuracy}")

accuracy:0.9535
